In [1]:
from tqdm import trange
from tqdm import tqdm
from urllib.request import urlopen, Request
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
import warnings
import pandas as pd
import sys

In [2]:
driver = webdriver.Chrome("C:/Untitled Folder/chromedriver_win32/chromedriver")
#인스터접속하여 로그인
def insta_login(login_id,login_pw):
    global driver  
    #인스타 접속
    driver.get('https://www.instargram.com')
    time.sleep(2) #시간을 넉넉히 설정합니다.
    
    #접속시 뜨는 보호경고 넘기기
        #고급 버튼 클릭
    high_class=driver.find_element_by_xpath("""//*[@id="details-button"]""")
    high_class.click()
    time.sleep(1)
        #페이지로이동하기 클릭
    page_move=driver.find_element_by_xpath("""//*[@id="proceed-link"]""")
    page_move.click()
    time.sleep(1)

    #로그인에 필요한 사항을 브라우저 제어를통해 입력합니다.
    email = login_id   ### 사용할 계정 정보 입력
    input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
    input_id.clear()
    input_id.send_keys(email)

    password = login_pw ### 비번 정보 수정
    input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
    input_pw.clear()
    input_pw.send_keys(password)
    input_pw.submit()
    
#url 주소에 해시테그 키워드적용
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

#첫게시물을 여는 함수를 정의합니다
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

#다음게시글을 여는 함수를 정의합니다.
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

#내용을 가져오는 함수를 정의합니다.
def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    

    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 정규표현식을 활용하여 해시태그를 가져옵니다.
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자를 가져옵니다.
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 수를 가져옵니다.
    try:
        like = driver.find_element_by_css_selector('div.Nm9Fw').text[4:-1] 
    except:
        like = 0
    # 위치정보를 가져옵니다.
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # 수집한 정보를 리스트로 저장합니다.
    data = [content, date, like, place, tags]
    return data

In [3]:
insta_login("id","pw")
time.sleep(1)

In [28]:
word = "비타민앰플" #검색할 해시태그. 띄어쓰기를 사용하면 안됩니다
url = insta_searching(word)
driver.get(url)
time.sleep(2)

In [29]:
# 총 게시물 숫자 불러오기
pageString = driver.page_source #현제 페이지의 htmlcode전체를 변수로에 저장
bsObj = BeautifulSoup(pageString, 'lxml') #BeautifulSoup 사용
temp_data = bsObj.find_all(name='meta')[-1]
temp_data = str(temp_data)
start = temp_data.find('게시물') + 4
end = temp_data.find('개')
total_data = temp_data[start:end]
print("총 {0}개의 게시물이 검색되었습니다.".format(total_data))

"""태그 크롤링"""
print("게시물을 수집하는 중입니다.")

총 22.8천개의 게시물이 검색되었습니다.
게시물을 수집하는 중입니다.


In [30]:
#스크롤링 작업
SCROLL_PAUSE_TIME = 1.0
reallink = []

while True:
    pageString = driver.page_source
    bsObj = BeautifulSoup(pageString, 'lxml')

    for link1 in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
        for i in range(3):
            title = link1.select('a')[i]
            real = title.attrs['href']
            reallink.append(real)

    last_height = driver.execute_script('return document.body.scrollHeight')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue
    # 게시물의 개수가 계속 바뀌거나 전체 게시물을 가져오지 못한다면
    # 아래 time.sleep의 시간(초)을 늘려 대기시간을 확보한다
    time.sleep(4)

In [31]:
# 전체 데이터 및 데이터 배치 사이즈 나누기
num_of_data = len(set(reallink))


print("검색된 {0}개의 게시물 중 {1}개의 게시물을 가지고오는데 성공하였습니다.".format(total_data, num_of_data))
print('총 {0}개의 데이터를 수집합니다.'.format(num_of_data))

검색된 22.8천개의 게시물 중 33개의 게시물을 가지고오는데 성공하였습니다.
총 33개의 데이터를 수집합니다.


In [222]:
set(reallink)

{'/p/BlFKCiXgOyc/',
 '/p/CIXUu0SjIp4/',
 '/p/CJ4o2cthJZz/',
 '/p/CJJFR-7HSTD/',
 '/p/CJUTL0FMge5/',
 '/p/CJvDnx1sa9J/',
 '/p/CK-T-_9Fnr2/',
 '/p/CK-UHWkjUBl/',
 '/p/CK05UPHF4YG/',
 '/p/CK05fYFFpR0/',
 '/p/CK05jsclVSf/',
 '/p/CK0ECBEFsR0/',
 '/p/CK0F3JQnZBS/',
 '/p/CK0Oq9tHUXU/',
 '/p/CK0t8ecn0vo/',
 '/p/CK26pLRHqAa/',
 '/p/CK2ZbKQD6rc/',
 '/p/CK2eBQmn5IJ/',
 '/p/CK2vD8PlPFx/',
 '/p/CK2x4yHFDD7/',
 '/p/CK37TTzLWKT/',
 '/p/CK3CkfRFUaN/',
 '/p/CK3Pcx_nHtb/',
 '/p/CK3cXPAnlu_/',
 '/p/CK3i7alF_8L/',
 '/p/CK3w44mHN-r/',
 '/p/CK3wIZ-H6xv/',
 '/p/CK3yIyLH-7i/',
 '/p/CK53eMFnJGX/',
 '/p/CK5ALBqFcve/',
 '/p/CK5FnlZlmdf/',
 '/p/CK5eenzF0Ve/',
 '/p/CK5t2ylpNCI/',
 '/p/CK6LP-5DX3v/',
 '/p/CK8TZotl2hm/',
 '/p/CK9Bc9llJY1/',
 '/p/CK9DtOPLsUn/',
 '/p/CK9NvS3nDfj/',
 '/p/CKD2NBiBPfF/',
 '/p/CKJCQjAhYeC/',
 '/p/CK_jlAOFGti/',
 '/p/CKvZvz0H6PO/',
 '/p/CKxKtfNs2IP/',
 '/p/CKxNhIajeeA/',
 '/p/CKxkEQrJw1Q/',
 '/p/CKxkF2Ll-Nn/',
 '/p/CKxuYX_F0SQ/',
 '/p/CKxzu9kg4rU/',
 '/p/CKy8JU6FXNO/',
 '/p/CKyF7Mnnr8j/',


In [223]:
len(set(reallink))

411

In [22]:
#함수를 차례대로 실행합니다.
select_first(driver)
results = [ ]
target = 515     # 크롤링할 게시글 수

for i in trange(target):
    # 게시글 수집에 오류 발생시 2초 대기후 다음글로 넘어갑니다.
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(1)
        move_next(driver)

100%|████████████████████████████████████████████████████████████████████████████████| 515/515 [27:52<00:00,  3.25s/it]


In [69]:
results

[["@reve_vcds #리그램 -백번 말로 좋다고 하는 것보다 직접 한번 써봐야 알겠죠? 일주일만에 달라지는 피부를 경험하실수 있게 준비했습니다.REVE 1기 체험단 모집!프리미엄 비타민C 세럼, 무료 체험할 기회 지금입니다!-* 기미,주근깨,잡티,트러블 때문에 고민이신분* 투명하고 깨끗한 피부를 원하시는분* 미백 화장품에 효과를 못보신분* 주름때문에 고민이신분* 피부에 생기가 없고 활렬이 떨어지신분-*모집기간 : 2021.1.19~2021.1.24-*모집인원 : 10명-*참여 방법1.@reve_vcds 공식 인스타그램 팔로우 하기2.본 게시물에 '좋아요♥' 누르기3.팔로우, 좋아요 모두 하셨다면 , 본 게시물 리그램하기-*본 게시물 댓글에 친구소환 시 체험단 선정 확률이 올라갑니다.*당첨자 발표는 1월 25일(일) 개별적 DM을 통해 발표-<체험단 미션>*제품 수령 후, 가이드 기한 내에 상품 리뷰 작성하기 (전/후 사진 필수)*남겨주신 리뷰는 추후 REVE의 마케팅 자료로 사용 될 수 있습니다.*체험단 미션을 수행 하실 수 있는 분들만

In [23]:
results_df = pd.DataFrame(results)
results_df.columns = ['content','date','like','place','tags']

In [24]:
results_df

,content,date,like,place,tags
0,#광고 쫀쫀탱탱 찐 콜라겐이 나타났다😵❤성분천재 바이애콤에서 드디어탄력/주름 케어템...,2021-02-08,0,,"[#광고, #콜라겐퍼밍앰플, #콜라겐퍼밍아이패치, #콜라겐퍼밍아이크림, #바이애콤,..."
1,해외에 메일정리해서 보내고 나니 시간이 ㅜㅜ중국어라 모라고 썼는지 알 수 없.......,2021-03-02,440,,"[#물광화장품, #물광메이크업, #수분광채에센스, #앰플추천, #앰플, #보르피린앰..."
2,🚨꺄 좋은소식 아시죠???이번엔 위드나미 사이트에서 구매가가능하세요! 이번 놓치시면...,2021-03-06,0,,"[#듀엠, #앰플, #기초화장품]"
3,#협찬엘리스랩 어썸 인텐시브 콜라겐 애...,2021-03-07,409,"Seoul, Korea",[#협찬엘리스랩]
4,지금 오직 #올리브영 에서만! 03.02 ~ 03.08 기간 한정 모든 티르티르 제...,2021-03-06,"12,507",,"[#올리브영, #극건성, #건성, #결광, #도자기크림, #수부지, #지성, #밀크..."
...,...,...,...,...,...
503,아니😳 누구신데 거기서 둥실둥실 떠다니시느...,2021-03-05,29,,[#트리플카밍앰플]
504,"‼️트리플랩 체험단 모집‼️⠀트리플랩에서 펀딩 예정 제품인 ""트리플 카밍 앰플"" 을...",2021-03-06,17,,"[#트리플랩, #트리플랩서포터즈, #트리플랩크루, #treeplelab, #트리플카..."
505,@koreanoritter さまから@klairs.jp の商品をモニターとして当選しいた...,2021-03-06,42,,[]
506,👉🏻로자티브 콜라스틴+쿠퍼로즈 mix👉🏻잔다셀 ᄆ...,2021-03-06,30,,[]


In [25]:
results_df.to_csv("data/insta/15_data.csv", mode='w',encoding='utf-8-sig')

In [ ]:
df["title"] = df["title"].str.replace(pat=r'[^\w]', repl=r'', regex=True)